如要爬一部份至少需要執行crawl_lst initialize的cell

In [1]:
import numpy as np
import pandas as pd

from urllib.parse import urlparse
import os

import requests as rq
from bs4 import BeautifulSoup
import io
import time

In [296]:
tb = pd.read_csv("tbrain_train_final_0610.csv")
tb_link = tb[["hyperlink"]]

f = lambda x: urlparse(x).scheme + "://" + urlparse(x).netloc
tb_link = tb_link.assign(baseUrl = tb_link["hyperlink"].map(f))
tb_link["baseUrl"].unique()

array(['https://news.cnyes.com', 'https://udn.com',
       'https://www.mirrormedia.mg', 'https://www.chinatimes.com',
       'http://domestic.judicial.gov.tw', 'http://www.coolloud.org.tw',
       'https://m.ctee.com.tw', 'https://mops.twse.com.tw',
       'https://www.hk01.com', 'https://www.wealth.com.tw',
       'https://news.ebc.net.tw', 'https://news.mingpao.com',
       'https://www.bnext.com.tw', 'http://news.ltn.com.tw',
       'http://finance.technews.tw', 'https://www.fsc.gov.tw',
       'https://news.tvbs.com.tw', 'https://www.cw.com.tw',
       'https://news.ltn.com.tw', 'https://www.businesstoday.com.tw',
       'https://sina.com.hk', 'https://www.ettoday.net',
       'https://hk.on.cc', 'http://technews.tw', 'https://money.udn.com',
       'https://tw.news.yahoo.com', 'https://www.setn.com',
       'https://www.managertoday.com.tw', 'https://www.cna.com.tw',
       'http://estate.ltn.com.tw', 'https://m.ltn.com.tw',
       'http://ccc.technews.tw', 'https://www.hbrtaiwan

In [5]:
crawl_baselst = ['http://technews.tw',
'https://money.udn.com',
'https://tw.news.yahoo.com',
'https://www.setn.com',
'https://www.managertoday.com.tw',
'https://www.cna.com.tw',
'http://estate.ltn.com.tw',
'https://m.ltn.com.tw',
'http://ccc.technews.tw']

In [6]:
# 根據base list 存成dict
crawl_lst = {}
for i in crawl_baselst:
    match_base = lambda x: i in x 
    crawl_lst[i] = tb_link["hyperlink"][tb_link["hyperlink"].map(match_base)].values

In [7]:
# corresponds with crawl list, storing all the article in different website
text_lst = {}

# http://technews.tw

In [171]:
## 隨便找一篇來看一下網址
crawl_lst['http://technews.tw'][2]
## 看看內文範圍到哪
tb.loc[tb_link.hyperlink == crawl_lst['http://technews.tw'][3]]

,news_ID,hyperlink,content,name
247,248,http://technews.tw/2019/11/26/chip-design-indu...,全球5G商轉進展可期 ### 省略內文 ### 時時更新科技新報粉絲團訂閱免費電子報,[]


In [217]:
## crawler
article_lst = []
for i in crawl_lst['http://technews.tw']:
    nl_response  = rq.get(i)
    soup = BeautifulSoup(nl_response.text, "lxml")
    soup = soup.find(class_="entry-content")
    text = ""
    for el in soup.find_all('p'):
        text += el.getText()
    article_lst.append(text)
text_lst['http://technews.tw'] = (article_lst, crawl_lst['http://technews.tw'])

# https://money.udn.com

In [173]:
## 隨便找一篇來看一下網址
crawl_lst['https://money.udn.com'][1]
## 看看內文範圍到哪
tb.loc[tb_link.hyperlink == crawl_lst['https://money.udn.com'][1]]

,news_ID,hyperlink,content,name
1105,1106,https://money.udn.com/money/story/5648/4109057,潤寅詐貸案越滾越大。檢調查出 ### 省略內文 ### 過程猶如連續劇。,"['謝春發', '王博民', '歐兆賢', '楊文虎', '黃明堂', '林寶菊', '蕭錦..."


In [215]:
## crawler
article_lst = []
for i in crawl_lst['https://money.udn.com']:
    try:
        nl_response  = rq.get(i)
        soup = BeautifulSoup(nl_response.text, "lxml")
        soup = soup.find(id = "article_body")
        soup.find_all('p')
        text = ""
        for el in soup.find_all('p'):
            text += el.getText()
        article_lst.append(text)
    except:
        print('unable to get ' + i)
        article_lst.append("NA")
text_lst['https://money.udn.com'] = (article_lst, crawl_lst['https://money.udn.com'])

unable to get https://money.udn.com/money/story/5621/3790042
unable to get https://money.udn.com/money/story/5648/3842152
unable to get https://money.udn.com/money/story/5612/3825917
unable to get https://money.udn.com/money/story/5648/3721782


# https://tw.news.yahoo.com

In [8]:
## 隨便找一篇來看一下網址
crawl_lst['https://tw.news.yahoo.com'][0]
## 看看內文範圍到哪
tb.loc[tb_link.hyperlink == crawl_lst['https://tw.news.yahoo.com'][0]]

,news_ID,hyperlink,content,name
208,209,https://tw.news.yahoo.com/%E5%BB%89%E6%94%BF-%...,曾在電視劇《廉政英雄》 ### 省略內文 ### 斷送自己的演藝路。,['陳淳伍']


In [218]:
## crawler
article_lst = []
for i in crawl_lst['https://tw.news.yahoo.com']:
    try:
        nl_response  = rq.get(i)
        soup = BeautifulSoup(nl_response.text, "lxml")
        soup = soup.find("article")
        p_lst = soup.find_all('p')
        text = ""
        for el in p_lst:
            # remove irrelevance
            if  "<p>\n <span>" in el.prettify():
                continue
            else:
                text += el.getText()
        article_lst.append(text)
    except:
        print('unable to get ' + i)
text_lst['https://tw.news.yahoo.com'] = (article_lst, crawl_lst['https://tw.news.yahoo.com'])

# https://www.setn.com

In [9]:
## 隨便找一篇來看一下網址
crawl_lst['https://www.setn.com'][1]
## 看看內文範圍到哪
tb.loc[tb_link.hyperlink == crawl_lst['https://www.setn.com'][1]]

,news_ID,hyperlink,content,name
295,296,https://www.setn.com/News.aspx?NewsID=629029,三立新聞／綜合報導刑事局日前才風光公布 ### 省略內文 ### 絕不寬待。,[]


In [219]:
## crawler
article_lst = []
for i in crawl_lst['https://www.setn.com']:
    try:
        nl_response  = rq.get(i)
        soup = BeautifulSoup(nl_response.text, "lxml")
        soup = soup.find(id = "Content1")
        p_lst = soup.find_all('p')
        text = ""
        for el in p_lst:
            # remove irrelevance
            text += el.getText()
        article_lst.append(text)
    except:
        print('unable to get ' + i)
text_lst['https://www.setn.com'] = article_lst
a = []
for i in text_lst['https://www.setn.com']:
    a.append(i[:-1])
text_lst['https://www.setn.com'] = (a, crawl_lst['https://www.setn.com'])

# https://www.managertoday.com.tw

In [94]:
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [37]:
## 隨便找一篇來看一下網址
crawl_lst['https://www.managertoday.com.tw'][0]
## 看看內文範圍到哪
tb.loc[tb_link.hyperlink == crawl_lst['https://www.managertoday.com.tw'][0]]

,news_ID,hyperlink,content,name
308,309,https://www.managertoday.com.tw/articles/view/...,「對於XXX遇害一事 ### 省略內文 ### 讓品牌聲譽一夕急轉直下。,[]


In [291]:
## crawler
article_lst = []
browser = webdriver.Chrome(ChromeDriverManager().install())
for i in crawl_lst['https://www.managertoday.com.tw']:
    try:
        browser.get(i)
        c = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.content.PostHtmlView"))
            )
        article_lst.append(c.text)
    except:
        print('unable to get ' + i)
        article_lst.append("NA")
text_lst['https://www.managertoday.com.tw'] = (article_lst, crawl_lst['https://www.managertoday.com.tw'])

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Driver [C:\Users\PaulYu\.wdm\drivers\chromedriver\win32\83.0.4103.39\chromedriver.exe] found in cache


In [155]:
#
#contents = browser.find_element_by_xpath("/html/body/div[8]/div/div/div/div/div[2]/div/div[1]/div/div/div[6]/div[1]/p[1]")
#contents

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[8]/div/div/div/div/div[2]/div/div[1]/div/div/div[6]/div[1]/p[1]"}
  (Session info: chrome=83.0.4103.116)


# https://www.cna.com.tw

In [53]:
## 隨便找一篇來看一下網址
crawl_lst['https://www.cna.com.tw'][0]
## 看看內文範圍到哪
tb.loc[tb_link.hyperlink == crawl_lst['https://www.cna.com.tw'][0]]

,news_ID,hyperlink,content,name
391,392,https://www.cna.com.tw/news/asoc/201906180229....,（中央社記者蕭博文台北18日電）高雄地方法院審理一件毒品走私案 ### 省略內文 ### 已...,"['張慶龍', '楊政錦']"


In [221]:
## crawler
article_lst = []
for i in crawl_lst['https://www.cna.com.tw']:
    try:
        nl_response  = rq.get(i)
        soup = BeautifulSoup(nl_response.text, "lxml")
        soup = soup.find(class_ = "paragraph")
        p_lst = soup.find_all('p')
        text = ""
        for el in p_lst:
            text += el.getText()
        article_lst.append(text)
    except:
        print('unable to get ' + i)
text_lst['https://www.cna.com.tw'] = (article_lst, crawl_lst['https://www.cna.com.tw'])

# http://estate.ltn.com.tw

In [77]:
## 隨便找一篇來看一下網址
crawl_lst['http://estate.ltn.com.tw'][0]
## 看看內文範圍到哪
tb.loc[tb_link.hyperlink == crawl_lst['http://estate.ltn.com.tw'][0]]

,news_ID,hyperlink,content,name
502,503,http://estate.ltn.com.tw/article/6814,位於桃園市桃園區春日路的建案「銘曜高尚」近日驚傳建商落跑 ### 省略內文 ### 才能確保...,"['邱俊銘', '邱水成', '邱秀芬']"


In [244]:
###
#
# todo: speed this up
#
###
## crawler
article_lst = []
browser = webdriver.Chrome(ChromeDriverManager().install())
for i in crawl_lst['http://estate.ltn.com.tw']:
    try:
        browser.get(i)
        WebDriverWait(browser, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.wordright"))
            )
        contents = browser.find_elements(By.XPATH, "/html/body/div[2]/div[9]/div[1]/div[3]/div[2]/p")

        for i, item in enumerate(contents):
            contents[i]  = item.text

        def nearEmptyString(txt):
            if txt != " ":
                return txt
        contents = list(filter(nearEmptyString, contents))
        article_lst.append("".join(contents[:-1]))
    except:
        print('unable to get ' + i)
text_lst['http://estate.ltn.com.tw'] = (article_lst, crawl_lst['http://estate.ltn.com.tw'])

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Driver [C:\Users\PaulYu\.wdm\drivers\chromedriver\win32\83.0.4103.39\chromedriver.exe] found in cache


# https://m.ltn.com.tw

In [65]:
## 隨便找一篇來看一下網址
crawl_lst['https://m.ltn.com.tw'][0]
## 看看內文範圍到哪
tb.loc[tb_link.hyperlink == crawl_lst['https://m.ltn.com.tw'][0]]

,news_ID,hyperlink,content,name
546,547,https://m.ltn.com.tw/news/society/breakingnews...,高市旗山警分局嶺口派出所前所長徐洪貴被控2件貪瀆案 ### 省略內文 ### 所以刑度比較重。,['徐洪貴']


In [223]:
## crawler
article_lst = []
for i in crawl_lst['https://m.ltn.com.tw']:
    try:
        browser.get(i)
        WebDriverWait(browser, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.boxTitle"))
            )
        contents = browser.find_elements(By.XPATH, "/html/body/div[4]/div[1]/div[4]/div[2]/p")
        for i, item in enumerate(contents):
            contents[i]  = item.text
        contents = list(filter(None, contents))
        article_lst.append("".join(contents[:-1]))
    except:
        print('unable to get ' + i)
text_lst['https://m.ltn.com.tw'] = (article_lst, crawl_lst['https://m.ltn.com.tw'])

# http://ccc.technews.tw

In [73]:
## 隨便找一篇來看一下網址
crawl_lst['http://ccc.technews.tw'][0]
## 看看內文範圍到哪
tb.loc[tb_link.hyperlink == crawl_lst['http://ccc.technews.tw'][0]]

,news_ID,hyperlink,content,name
613,614,http://ccc.technews.tw/2019/07/03/samsung-gala...,中國IT之家報導 ### 省略內文 ### 時時更新科技新報粉絲團訂閱免費電子報,[]


In [224]:
article_lst = []
for i in crawl_lst['http://ccc.technews.tw']:
    try:
        nl_response  = rq.get(i)
        soup = BeautifulSoup(nl_response.text, "lxml")
        soup = soup.find(class_ = "indent")
        p_lst = soup.find_all('p')
        text = ""
        for el in p_lst:
            text += el.getText()
        article_lst.append(text)
    except:
        print('unable to get ' + i)
text_lst['http://ccc.technews.tw'] = (article_lst, crawl_lst['http://ccc.technews.tw'])

# to csv

In [292]:
text_lst_txt = [i[0] for i in text_lst.values()]
text_lst_link = [i[1] for i in text_lst.values()]
text_lst_link = np.hstack(text_lst_link)
text_lst_txt = np.hstack(text_lst_txt)
df = pd.DataFrame(data = {'crawled.text': text_lst_txt, 'hyperlink': text_lst_link})
news_Paul = df.merge(tb, on='hyperlink', how='left')

In [302]:
news_Paul.to_csv(index=False)

'crawled.text,hyperlink,news_ID,content,name\r\n社會中心／綜合報導高雄近日爆發「直播主之亂」，知名直播主連千毅與寵物店老闆兼直播主鄭又仁槓上，引發砸店、開槍風波，昨日雙雙落網，今早分別進行偵訊。如今傳出連千毅直播販售的「賓士行李箱」疑為假貨，高雄檢警20日前往倉庫搜索將證物帶回，連千毅連忙開直播澄清，強調自己是代銷商，更秀出簡體字授權函，強調絕對是正品，但卻遭台灣賓士打臉，直指「德國奔馳戴姆勒公司」不存在！▲連千毅如今傳出賣假貨疑雲。（圖／翻攝臉書）連千毅又被封為「直播天王」，短短幾分鐘就能賣出跑車、直升機等，成功在業界闖出一片天，多次登上媒體版面，但卻因行事作風太高調，引來黑道注目，不僅衝突頻傳，如今更傳出他所賣的「賓士行李箱」是假貨。高雄檢警20日上午會同保二總隊、經發局等單位，聯合稽查搜索連千毅的倉庫直播站，當場查扣一些賓士行李箱等商品，疑似有仿冒嫌疑，將其帶回調查，連千毅則大動作維護商譽，強調自己是代銷商，「全亞洲不是只有你賣賓士！」、「今天台灣授權廠商有講，未經他們認同的都非正貨，路上10台有8台是水貨車，你能說他是山寨仿冒嗎？」並揚言要對不實媒體提告。▲連千毅強調自己只是代銷商。（圖／翻攝臉書）連千毅秀出賓士授權函，強調產品是正品，但卻有眼尖網友發現疑點，除了中文部分皆用簡體字外，連授權書的英文都拼錯，將「AUTHORIZATION」打成「AUTHORIZAT『T』ON」，此外授權方「茲」也不知道是誰，台灣賓士對此回應：「我們戴姆勒集團旗下子公司，沒有一個叫做『德國奔馳戴姆勒公司』，並沒有這樣的公司存在。」據《ETtoday》報導，追查同款式行李箱，發現連千毅在直播上喊價拍賣上，知名購物平台也有同款商品，要價3萬9千元，但連千毅卻從6000元開始喊價，價差相當之大，如今傳出該款行李箱為仿冒品後，商品已從該網站悄悄下架，避免衍生風波。▲授權書被抓包不僅是簡體字，就連英文的授權書都打錯。（圖／翻攝連千毅臉書）,https://www.setn.com/News.aspx?NewsID=606443,225,高雄近日爆發「直播主之亂」 ### 省略內文 ### 避免衍生風波。,"[\'連千毅\', \'鄭又仁\']"\r\n三立新聞／綜合報導刑事局日前才風光公布，首次和民間跨界合作，拍攝2020年的月曆，沒想到其